In [58]:
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris, load_digits

from ipywidgets import interact

In [59]:
np.random.seed(1024)

In [62]:
dataset = load_digits()
# dataset = load_iris()
X, y = dataset.data, dataset.target
# X = X / 255
X = StandardScaler().fit_transform(X)

In [63]:
@interact(method=['auto', 'full', 'arpack', 'randomized'])
def do_pca(method):
    pca = PCA(n_components=2, svd_solver=method, random_state=1024)
    print(pca)
    X2d = pca.fit_transform(X)
    plt.scatter(X2d[:,0], X2d[:,1], c=y)
    plt.matshow(pca.components_, cmap='viridis')
    plt.show()

interactive(children=(Dropdown(description='method', options=('auto', 'full', 'arpack', 'randomized'), value='…